In [1]:
from choice_model import ChoiceModel
import networkx as nx
model = ChoiceModel()

In [2]:
period = 0
example_input = {
'age': '18-35',
'gender': 'Female',
'marrige': 'Yes',
'withkids': 'No',
'student': 'Yes',
'workstatue': 'Freelancer',
'residentinneighbor': 'No',
'educationlevel': 'Undergraduate'
}
k1 = 10
k2 = 5
model.set_period(period)

In [3]:
similar_nodes,similar_scores = model.similarity_search(example_input, k1, k2)
print("Similar nodes: ", similar_nodes) 
print("Scores: ", similar_scores)

Similar nodes:  [12, 469, 539, 154, 575, 457, 768, 772, 513, 1079, 686, 702, 705, 714, 725, 681, 464, 373, 49, 603, 573, 683, 698, 700, 709]
Scores:  [0.49987637996673584, 0.49987637996673584, 0.49987637996673584, 0.49987637996673584, 0.49987637996673584, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.6176310777664185, 0.7890002727508545, 0.7890002727508545, 0.7890002727508545, 0.7890002727508545, 0.7890002727508545]


# Create Subgraph of similar nodes and their neighbors

In [4]:
neighbors = []
for n in similar_nodes:
    neighbors.extend(list(model.graph.neighbors(n)))
    neighbors.append(n) 
sub_graph = nx.subgraph_view(model.graph, filter_node=lambda n: n in neighbors)

In [5]:
neighbors = list(set(neighbors))
print("Neighbors: ", neighbors)

Neighbors:  [768, 513, 772, 1030, 1035, 12, 1037, 270, 662, 154, 1050, 539, 1055, 1056, 681, 811, 683, 686, 49, 181, 1079, 698, 700, 1085, 702, 575, 61, 705, 573, 1091, 1092, 963, 964, 709, 457, 970, 714, 1097, 464, 592, 469, 982, 725, 217, 603, 994, 995, 226, 756, 373, 1022]


# Add Predict Node to the Subgraph

In [6]:
compute_graph = sub_graph.copy()

In [7]:
# normalize the weights of links
weights = nx.get_edge_attributes(compute_graph, 'weight')
max_weight = max(weights.values())
for e in compute_graph.edges():
    compute_graph[e[0]][e[1]]['weight'] = compute_graph[e[0]][e[1]]['weight'] / max_weight

# normalize the weights of similarity scores
max_score = max(similar_scores)
similar_scores = [s / max_score for s in similar_scores]

In [8]:
import uuid
agent_id = uuid.uuid4().hex
compute_graph.add_node(
    agent_id, 
    type='Agent', 
    period = 1, 
    properties=example_input,
    label='Agent'
)
for node,scocre in zip(similar_nodes,similar_scores):
    compute_graph.add_edge(agent_id, node, weight=scocre, type='similar_to',label='similar_to') 

In [9]:
compute_graph.nodes()

NodeView((681, 457, 464, 12, 573, 469, 217, 539, 154, 373, 683, 575, 686, 49, 698, 700, 702, 705, 709, 714, 725, 756, 768, 772, 603, 61, 964, 811, 963, 970, 982, 994, 995, 1022, 1030, 662, 1035, 1037, 1050, 513, 1055, 1056, 181, 1079, 1085, 1091, 1092, 1097, 592, 226, 270, '075c8adfca3e41709a73d509fefa8114'))

# Visualization

In [10]:
import pyvis.network as net

nx_graph = compute_graph.copy()
nt = net.Network("800px", "1200px")
nt.from_nx(nx_graph=nx_graph)
nt.write_html("example.html")

# Link Prediction

In [31]:
import numpy as np
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def cal_weighted_jaccard_coeff(G,agent,choice):
    G_ = G.to_undirected()
    common_neighbors = list(nx.common_neighbors(G_,agent,choice))
    total_neighbors = len(list(nx.neighbors(G_,agent)))
    weight_sum = 0
    for n in common_neighbors:
        edge_ap = G_.edges[agent,n]['weight']
        edge_pc = G_.edges[n,choice]['weight']
        weight_sum += (edge_ap+edge_pc)/2
    return weight_sum/total_neighbors

def get_recommendation(G,choice_type='Actors',top_k=3):
    agents = [n for n,d in G.nodes(data=True) if d['type'] == 'Agent']
    choices = [n for n,d in G.nodes(data=True) if d['type'] == choice_type]
    recommendation = {}
    for choice in choices:
        recommendation[choice] = cal_weighted_jaccard_coeff(G,agents[0],choice)
    max_weight = max(recommendation.values())
    recommendation = {k: v/max_weight for k, v in sorted(recommendation.items(), key=lambda item: item[1],reverse=True)}
    choices = list(recommendation.keys())
    choices = [G.nodes[n] for n in choices]
    probabilities = softmax(list(recommendation.values())[:top_k]).tolist()
    return choices,probabilities

In [32]:
choices,recommendation = get_recommendation(compute_graph,choice_type='Media Content')

In [33]:
recommendation

[0.36624763428173485, 0.31687618285913255, 0.31687618285913255]